#### CNN Model:

In [50]:
# libraries cnn model
import pandas as pd

import numpy as np
from numpy import mean
from numpy import std
from tensorflow import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from sklearn.metrics import (confusion_matrix, accuracy_score, classification_report)

import plotly.figure_factory as ff

import pickle
import os
import scipy.io as sio
from scipy.fft import fft

In [42]:
# Loading functions for all files in a certain configuration
# load a single file as a numpy array
def load_file(filepath, filename):
	data = sio.loadmat(filepath)
	
	# vibration data
	vib_data = data[filename]['Y'][0][0][0][6][2].transpose()
	vib_data = vib_data.reshape(vib_data.shape[0],)

	# frequency domain and splitting into samples of size 1024 (512 in freq)
	n_loop = round(vib_data.shape[0]/1024)
	vib_freq = np.zeros([n_loop,512])
	for i in range(n_loop):
		vib_data_1024 = vib_data[1024*i:1024*(1+i)]
		vib_freq[i] = np.abs(fft(vib_data_1024))[0:1024//2]

	return vib_freq

# load a dataset group, such as train or test
def load_dataset_group(prefix='',group='', setting=''):
	filepath = prefix + group + '/' + setting

	# load all 20 files as a single array
	# X will have 20*N rows and 512 columns
	# where N is the number of rows of each file when splitting into samples
	X = np.zeros([1,512])
	for i in range(1,4): # changed 20 to 3 to reduce dataset length
		path = filepath+'_'+group+'_'+str(i)+'.mat'
		filename = setting+'_'+group+'_'+str(i)
		if i == 1:
			X = load_file(path,filename)
		else:
			X = np.concatenate([X,load_file(path,filename)],axis=0)
	
	# load class output
	if group=='K001' or group=='K002' or group=='K003' or group=='K004' or group=='K005' or group=='K006': # healthy
		y = np.ones(X.shape[0])
	elif group=='KA04' or group=='KA16' or group=='KA22':# real OR damage
		y = np.ones(X.shape[0])*2
	elif group=='KI04' or group=='KI16' or group=='KI18':# real IR damage
		y = np.ones(X.shape[0])*3
	elif group=='KA01' or group=='KA03' or group=='KA07':# artificial OR damage
		y = np.ones(X.shape[0])*2
	elif group=='KI01' or group=='KI03' or group=='KI07':# artificial IR damage
		y = np.ones(X.shape[0])*3
	else:
		y = np.zeros(X.shape[0])
	return X, y

# load the dataset, returns train and test X and y elements
def load_dataset_settings(prefix='',setting='N15_M07_F10'):

	# load all data
	X_k001, y_k001 = load_dataset_group(prefix+'PPDataset/', 'K001', setting) # healthy
	X_k002, y_k002 = load_dataset_group(prefix+'PPDataset/', 'K002', setting) # healthy

	X_ka01, y_ka01 = load_dataset_group(prefix+'PPDataset/', 'KA01', setting) # artificial OR damage

	X_ki01, y_ki01 = load_dataset_group(prefix+'PPDataset/', 'KI01', setting) # artificial IR damage

	X_ka04, y_ka04 = load_dataset_group(prefix+'PPDataset/', 'KA04', setting) # real OR damage

	X_ki04, y_ki04 = load_dataset_group(prefix+'PPDataset/', 'KI04', setting) # real IR damage

	X = np.concatenate([X_k001,X_k002,X_ka01,X_ki01,X_ka04,X_ki04])
	y = np.concatenate([y_k001,y_k002,y_ka01,y_ki01,y_ka04,y_ki04])

	# zero-offset class values
	y = y - 1
	# one hot encode y
	y = to_categorical(y)

	X = X.reshape(X.shape[0],X.shape[1],1)

	print(X.shape, y.shape)
	return X, y

In [19]:
# load the dataset, returns train and test X and y elements
def load_dataset():

	# load all data
	with open('PPDataset/train_test_data/trainX.npy', 'rb') as f:
		trainX = np.load(f)
	with open('PPDataset/train_test_data/trainy.npy', 'rb') as f:
		trainy = np.load(f)
	with open('PPDataset/train_test_data/testX.npy', 'rb') as f:
		testX = np.load(f)
	with open('PPDataset/train_test_data/testy.npy', 'rb') as f:
		testy = np.load(f)
	
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	
	return trainX, trainy, testX, testy

In [27]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 10, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(Conv1D(filters=4, kernel_size=3, activation='relu',input_shape=(n_timesteps,n_features)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=8, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    pred_train = model.predict(trainX, verbose=0)
    pred_test = model.predict(testX, verbose=0)
    
    # save model
    models_dir = 'models/'
    existing_models = [filename for filename in os.listdir(models_dir) if filename.startswith('cnn_model')]
    num_model = len(existing_models)+1
    filename = f'models/cnn_model_{num_model}.h5'
    model.save(filename)

    return accuracy, pred_train, pred_test

In [28]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [29]:
trainX, trainy, testX, testy = load_dataset()

(3022, 512, 1) (3022, 3) (1489, 512, 1) (1489, 3)


In [30]:
pd.DataFrame(np.argmax(trainy,axis=1)).value_counts()

1    1023
2    1017
0     982
dtype: int64

In [31]:
pd.DataFrame(np.argmax(testy,axis=1)).value_counts()

0    518
2    487
1    484
dtype: int64

In [32]:
# run an experiment
def run_experiment(repeats=10):
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	train_accs = list()
	test_accs = list()
	for r in range(repeats):
		score, pred_train, pred_test = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
		train_acc = accuracy_score(np.argmax(trainy,axis=1), np.argmax(pred_train,axis=1))*100
		test_acc = accuracy_score(np.argmax(testy,axis=1), np.argmax(pred_test,axis=1))*100
		train_accs.append(train_acc)
		test_accs.append(test_acc)

	# summarize results
	summarize_results(scores)
	print('Train accuracy: ')
	summarize_results(train_accs)
	print('Test accuracy: ')
	summarize_results(test_accs)


In [33]:
# run the experiment
run_experiment(10)

# me ha tardado 1m 5.7s pero ha bajado el accuracy bastante

(3022, 512, 1) (3022, 3) (1489, 512, 1) (1489, 3)
>#1: 97.918
>#2: 97.112
>#3: 96.776
>#4: 98.120
>#5: 97.918
>#6: 98.522
>#7: 96.373
>#8: 97.179
>#9: 97.582
>#10: 97.381
[97.9180634021759, 97.11215496063232, 96.7763602733612, 98.11954498291016, 97.9180634021759, 98.5224962234497, 96.37340307235718, 97.17931747436523, 97.58226871490479, 97.38079309463501]
Accuracy: 97.488% (+/-0.620)
Train accuracy: 
[99.9669093315685, 100.0, 99.04037061548642, 100.0, 99.9669093315685, 100.0, 100.0, 100.0, 100.0, 100.0]
Accuracy: 99.897% (+/-0.286)
Test accuracy: 
[97.91806581598388, 97.11215580926796, 96.77635997313632, 98.11954331766286, 97.91806581598388, 98.52249832102082, 96.37340496977838, 97.1793149764943, 97.58226997985226, 97.38079247817328]
Accuracy: 97.488% (+/-0.620)


In [43]:
# load data
trainX, trainy, testX, testy = load_dataset()

score, pred_train, pred_test = evaluate_model(trainX, trainy, testX, testy)
score = score * 100.0
train_acc = accuracy_score(np.argmax(trainy,axis=1), np.argmax(pred_train,axis=1))
test_acc = accuracy_score(np.argmax(testy,axis=1), np.argmax(pred_test,axis=1))
print('Train accuracy: ',train_acc)
print('Test accuracy: ',test_acc)

# Me ha tardado 8.5s, training accuracy muy bueno, pero test peor

(3022, 512, 1) (3022, 3) (1489, 512, 1) (1489, 3)
Train accuracy:  1.0
Test accuracy:  0.9785090664875755


In [44]:
print(classification_report(np.argmax(testy,axis=1), np.argmax(pred_test,axis=1), target_names=['Healthy', 'OR fault', 'IR fault'],digits=4))

              precision    recall  f1-score   support

     Healthy     0.9637    0.9749    0.9693       518
    OR fault     1.0000    1.0000    1.0000       484
    IR fault     0.9730    0.9610    0.9669       487

    accuracy                         0.9785      1489
   macro avg     0.9789    0.9786    0.9787      1489
weighted avg     0.9785    0.9785    0.9785      1489



In [45]:
# training
# Construimos una visualización para la matriz de confusión
z_train = confusion_matrix(np.argmax(trainy,axis=1), np.argmax(pred_train,axis=1))
# Reformateo la matriz para que me quede mejor el gráfico
z_train[[0,2],:] = z_train[[2,0],:]
x = ['Healthy', 'OR fault', 'IR fault']
y = ['IR fault', 'OR fault', 'Healthy']
z_text = [[str(y) for y in x] for x in z_train]
heatmap = ff.create_annotated_heatmap(z_train, x=x, y=y, annotation_text=z_text, colorscale='tealrose')
heatmap.update_layout(title_text='Training',height=300,width=600,
                      xaxis_title="Predicted Label",yaxis_title="True Label")
heatmap.show()

In [46]:
# testing
# Construimos una visualización para la matriz de confusión
z_test = confusion_matrix(np.argmax(testy,axis=1), np.argmax(pred_test,axis=1))
# Reformateo la matriz para que me quede mejor el gráfico
z_test[[0,2],:] = z_test[[2,0],:]
x = ['Healthy', 'OR fault', 'IR fault']
y = ['IR fault', 'OR fault', 'Healthy']
z_text = [[str(y) for y in x] for x in z_test]
heatmap = ff.create_annotated_heatmap(z_test, x=x, y=y, annotation_text=z_text, colorscale='tealrose')
heatmap.update_layout(title_text='Testing',height=300,width=600,
                      xaxis_title="Predicted Label",yaxis_title="True Label")
heatmap.show()

In [47]:
indicesIRHealthy = np.where((np.argmax(testy,axis=1) == 2) & (np.argmax(pred_test,axis=1) == 0))[0]
testX_IRHealthy = testX[indicesIRHealthy]
testX_IRHealthy = testX_IRHealthy.reshape(testX_IRHealthy.shape[0],testX_IRHealthy.shape[1])
testX_IRHealthy.shape

(19, 512)

In [48]:
# load all data
X_ki01, y_ki01 = load_dataset_group('PPDataset/', 'KI01', 'N15_M07_F10') # artificial IR damage
X_ki04, y_ki04 = load_dataset_group('PPDataset/', 'KI04', 'N15_M07_F10') # real IR damage

In [49]:
matches_artificial = []
for i in range(testX_IRHealthy.shape[0]):
    for j in range(X_ki01.shape[0]):
        if(np.array_equal(testX_IRHealthy[i], X_ki01[j])):
            matches_artificial.append(True)
            break
print("Artificial IR damage mistaken for Healthy: ",len(matches_artificial))

matches_real = []
for i in range(testX_IRHealthy.shape[0]):
    for j in range(X_ki04.shape[0]):
        if(np.array_equal(testX_IRHealthy[i], X_ki04[j])):
            matches_real.append(True)
            break
print("Real IR damage mistaken for Healthy: ",len(matches_real))

Artificial IR damage mistaken for Healthy:  15
Real IR damage mistaken for Healthy:  4


In [52]:
# Changing the rotational speed from 1500 rpm to 900 rpm
cnn_model = keras.models.load_model('models/cnn_model_13.h5')

X_N09_M07_F10, y_N09_M07_F10 = load_dataset_settings(prefix='',setting='N09_M07_F10')
pred_N09_M07_F10 = cnn_model.predict(X_N09_M07_F10)
acc_N09_M07_F10 = accuracy_score(np.argmax(y_N09_M07_F10,axis=1), np.argmax(pred_N09_M07_F10,axis=1))
print('Test accuracy: ',acc_N09_M07_F10)

(4501, 512, 1) (4501, 3)
141/141 [==============================] - 1s 6ms/step
Test accuracy:  0.6225283270384359


In [53]:
# Changing the load torque from 0.7 Nm to 0.1 Nm
X_N15_M01_F10, y_N15_M01_F10 = load_dataset_settings(prefix='',setting='N15_M01_F10')
pred_N15_M01_F10 = cnn_model.predict(X_N15_M01_F10)
acc_N15_M01_F10 = accuracy_score(np.argmax(y_N15_M01_F10,axis=1), np.argmax(pred_N15_M01_F10,axis=1))
print('Test accuracy: ',acc_N15_M01_F10)

(4507, 512, 1) (4507, 3)
141/141 [==============================] - 1s 6ms/step
Test accuracy:  0.9747060128688706


In [54]:
# Changing the radial force from 1000 N to 400 N
X_N15_M07_F04, y_N15_M07_F04 = load_dataset_settings(prefix='',setting='N15_M07_F04')
pred_N15_M07_F04 = cnn_model.predict(X_N15_M07_F04)
acc_N15_M07_F04 = accuracy_score(np.argmax(y_N15_M07_F04,axis=1), np.argmax(pred_N15_M07_F04,axis=1))
print('Test accuracy: ',acc_N15_M07_F04)

(4510, 512, 1) (4510, 3)
141/141 [==============================] - 1s 5ms/step
Test accuracy:  0.962749445676275
